In [282]:
#!/usr/bin/python3

from login_details import engine1

In [ ]:
def delete_rows_from_db(list_of_faults_delete):
    # Convert delete list to tuples. This allows to pass delete list into the SQL script
    list_of_faults_delete = tuple(list_of_faults_delete)

    params = {'l': list_of_faults_delete}
    with engine1.connect() as con:
        print("Keep List:")
        print(', '.join(map(str, list_of_faults_keep)))

        con.execute("""
            DELETE FROM dg_ln_stoppages_raw_1
            WHERE id in %(l)s;
            """, params)
        con.close()
        print(len(list_of_faults_delete))
        print('DB Updated!\n')

In [278]:
# get data from db
with engine1.connect() as con:
    query = con.execute("""
    SELECT id, stoppage_id, datetime
    FROM dg_ln_stoppages_raw_1
    ORDER BY datetime ASC
    """)
    con.close()

In [279]:
# setting value for the first iteration of the variable previous_row
previous_row = -1  # -1 only indicates that the variable hasn't been set - first iteration
list_of_faults_keep = []
list_of_faults_delete = []

In [280]:
# iterate through the database rows
for row in query:
    # previous_row needs a value for first iteration. After this, the value -1 will be overwritten by the actual row
    # value
    if previous_row == -1:  # first iteration
        previous_row = row[1]  # save the FAULT ID value
        list_of_faults_keep.append(row[1])

    elif row[1] == previous_row:  # repeating value - needs to be deleted
        list_of_faults_delete.append(row[0])  # save the row ID into delete list

        if len(list_of_faults_delete) > 5000:  # avoiding too big delete list as it might crash the run
            delete_rows_from_db(list_of_faults_delete)
            list_of_faults_delete = []

        previous_row = row[1]  # save the FAULT ID value

    elif row[1] != previous_row:  # the value is different from the previous row (change of state)
        if row[1] in list_of_faults_keep:
            if row[1] == 0:
                if len(list_of_faults_keep) > 1:
                    if list_of_faults_delete:  # Check if delete list has any values in it. If yes, initiate delete
                        # order. This check is to get past already sorted rows

                        # execute SQL sequence to delete rows from delete list
                        delete_rows_from_db(list_of_faults_delete)

                        list_of_faults_delete = []  # empty delete list
                        list_of_faults_keep = [row[1]]  # add last 0 to the list, so it won't keep the next one.
                        previous_row = row[1]  # save the FAULT ID value

                        continue  # return to beginning of the loop

                    else:  # if delete list is empty, reset the keep list. This is to get past already sorted rows
                        list_of_faults_keep = [row[1]]
                        continue

            if len(list_of_faults_delete) > 5000:  # avoiding too big delete list as it might crash the run
                delete_rows_from_db(list_of_faults_delete)
                list_of_faults_delete = []

            list_of_faults_delete.append(row[0])
            previous_row = row[1]  # save the FAULT ID value

        elif row[1] not in list_of_faults_keep:
            list_of_faults_keep.append(row[1])
            previous_row = row[1]  # save the FAULT ID value

if list_of_faults_delete:  # check if list has values. It would fail without any values in it.
    delete_rows_from_db(list_of_faults_delete)

engine1.dispose()  # properly close database connection to prevent memory leaks.

Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done


In [281]:
#print("Keep List:\n")
#print('\n'.join(map(str, list_of_faults_delete)))